# Project overview: 
## You are working for a property management company renting rooms and properties for short periods of time on various platforms. You need to estimate the typical price for a given property based on the price of similar properties. 

## We need to conduct the Exploratory Data Analysis (EDA) in order to understand the input sample data set

In [25]:
#!pip install pandas_profiling --upgrade

In [1]:
import yaml
import wandb
import numpy as np
import pandas as pd
import pandas_profiling
#import ydata_profiling
from IPython.display import HTML

/home/tta/miniconda3/envs/udacity/lib/python3.8/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/tta/miniconda3/envs/udacity/lib/python3.8/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
/tmp/ipykernel_397690/3942390920.py:5: DeprecationWarning: `import pandas_profiling` is going to be deprecated 

## Initliaze a project="nyc_airbnb" in Weights & Biases (W&B) and the notebook is uploaded and versioned in W&B
## Download the latest version of the sample.csv
## Read sample data file as the Pandas dataframes

In [2]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: tta123 (thanhta123). Use `wandb login --relogin` to force relogin


# Check the schema (or data structure of sample.csv)

In [3]:
# Data dimension
df.shape

(20000, 16)

In [4]:
# Data Columns and their corresponging data types as well as how many non null values per column
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20000 non-null  int64  
 1   name                            19993 non-null  object 
 2   host_id                         20000 non-null  int64  
 3   host_name                       19992 non-null  object 
 4   neighbourhood_group             20000 non-null  object 
 5   neighbourhood                   20000 non-null  object 
 6   latitude                        20000 non-null  float64
 7   longitude                       20000 non-null  float64
 8   room_type                       20000 non-null  object 
 9   price                           20000 non-null  int64  
 10  minimum_nights                  20000 non-null  int64  
 11  number_of_reviews               20000 non-null  int64  
 12  last_review                     

In [5]:
# Review the first few samples of the input data
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5
1,31444015,TIME SQUARE CHARMING ONE BED IN HELL'S KITCHEN...,8523790,Johlex,Manhattan,Hell's Kitchen,40.76682,-73.98878,Entire home/apt,170,3,0,NaN,NaN,1,188
2,8741020,Voted #1 Location Quintessential 1BR W Village...,45854238,John,Manhattan,West Village,40.73631,-74.00611,Entire home/apt,245,3,51,2018-09-19,1.12,1,0
3,34602077,Spacious 1 bedroom apartment 15min from Manhattan,261055465,Regan,Queens,Astoria,40.76424,-73.92351,Entire home/apt,125,3,1,2019-05-24,0.65,1,13
4,23203149,Big beautiful bedroom in huge Bushwick apartment,143460,Megan,Brooklyn,Bushwick,40.69839,-73.92044,Private room,65,2,8,2019-06-23,0.52,2,8


In [6]:
#from ydata_profiling import ProfileReport

In [10]:
from pathlib import Path
config_file_path = Path('../../config.yaml')

## Create a profile report of the sample data set and display the report results using the to_widgets method

In [11]:
#profile = pandas_profiling.ProfileReport(df, title="NYC Airbnb Profile report")
profile = pandas_profiling.ProfileReport(df, title="NYC Airbnb Profile report", config_file=config_file_path)
#profile.to_widgets()

#HTML(profile.to_html())
profile.to_file("./reports/Pandas-Profiling-Report—NYC-Airbnb.html")

ValidationError: 4 validation errors for Settings
data_check
  extra fields not permitted (type=value_error.extra)
etl
  extra fields not permitted (type=value_error.extra)
main
  extra fields not permitted (type=value_error.extra)
modeling
  extra fields not permitted (type=value_error.extra)

## Cleaning the data Process. such as:
## we need to carry out data cleaning processes such as: removing outliers and correcting data formats. 
## To remove outliers in the price column: 
## After talking to our stakeholders, we decide to consider only from a minimum of 10 dollars to 350 dollars per night. 
## We also convert the data type of "last_review" as date time

In [8]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [9]:
# check to see if the data is clean ?
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  number_

In [10]:
# Terminate the run
run.finish()